In [1]:
collection = "tweets_fernandezfernandez"

In [2]:
import json
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 999)

In [4]:
from pymongo import MongoClient

MONGO_HOST = 'mongodb://localhost/twitterdata'
client = MongoClient(MONGO_HOST)
db = client.twitterdata

In [5]:
coll = db[collection]

In [6]:
coll.estimated_document_count()

115690

In [7]:
# %%time
tweets = []
users = {}
cols_tweet = [
    'id',
    'created_at',
    'text',
    'favorited',
    'retweeted',
]

cols_user = [
    'screen_name',
    'location',
    'followers_count',
    'statuses_count',
    'description',
    'friends_count',
    'favourites_count',
    'created_at',
    'id',
    'name',             
    'verified'
]

# for t in coll.find().limit(10):
for t in coll.find():
    uid = t['user']['id'] 
    if uid not in users:
        users[uid] = [t['user'][c] for c in cols_user]
        
    tr = [t[c] for c in cols_tweet] + [t['user']['name'], t['user']['id']]

    mentions = " ".join([h['screen_name'] for h in t['entities']['user_mentions']])
    tr.append(mentions)
    
    hts = " ".join([h['text'] for h in t['entities']['hashtags']])
    tr.append(hts)

    if 'retweeted_status' in t:
        t = t['retweeted_status']
        
    tr.append(t['id'])
    if 'extended_tweet' in t:
        tr.append(t['extended_tweet']['full_text'])   
    else:
        tr.append(t['text'])

    tr += [t['favorite_count'], t['retweet_count']]

        
    tweets.append(tr)

In [8]:
tweets_df = pd.DataFrame(tweets, columns=cols_tweet + ['user_name', 'user_id', 'mentions', 'hts', 'original_id', 'full_text', 'favorite_count', 'retweet_count'
])

In [9]:
users_df = pd.DataFrame(users.values(), columns=cols_user)

## Guardar datos en formato accesible

In [10]:
clean_text = lambda s: s.replace('\n', ' ').replace('\r', ' ')

tweets_df.full_text = tweets_df.full_text.apply(clean_text)

tweets_df.text = tweets_df.text.apply(clean_text)

In [11]:
tweets_df.shape

(115690, 13)

In [12]:
tweets_df.head()

,id,created_at,text,favorited,retweeted,user_name,user_id,mentions,hts,original_id,full_text,favorite_count,retweet_count
0,1129796443533905921,Sat May 18 17:10:37 +0000 2019,RT @NuncaMasK2019: ALBERTO FERNÁNDEZ sobre CRISTINA KIRCHNER Una persona sin valores actúa siguiendo su ambición de poder y dinero. ¡Dale…,False,False,silvina,218159646,NuncaMasK2019,,1129754910030016515,ALBERTO FERNÁNDEZ sobre CRISTINA KIRCHNER Una persona sin valores actúa siguiendo su ambición de poder y dinero. ¡Dale RT para DIFUNDIR! #ArrugoCristina https://t.co/jeZO9zvSNF,836,951
1,1129796444729368576,Sat May 18 17:10:37 +0000 2019,"RT @felipe_sola: Mi candidatura tenía una única misión: construir una unidad amplia para defender al trabajo, a los humildes y a la clase m…",False,False,Tonga,144904500,felipe_sola,,1129761247359057922,"Mi candidatura tenía una única misión: construir una unidad amplia para defender al trabajo, a los humildes y a la clase media. Ese objetivo está cumplido. Celebro la decisión de Cristina. Felicito a Alberto. #EsConTodos",14194,3065
2,1129796445991837696,Sat May 18 17:10:37 +0000 2019,"180 mil millones de dólares de deuda en cimientos, ahora hay que garparlos Pinedo, contate otro.… https://t.co/3tHk0FIwCf",False,False,Guillermo Sordo,319594285,,,1129796445991837696,"180 mil millones de dólares de deuda en cimientos, ahora hay que garparlos Pinedo, contate otro. #FernandezFernandez #CristinaSomosTodos",0,0
3,1129796448718065666,Sat May 18 17:10:38 +0000 2019,"RT @peppa1963: @rag_HLVS Se lo van a cargar al pobre títere de Alberto Fernández.... ""Que parezca un accidente......"" #ArrugoCristina #Buen…",False,False,Silvia Berth,837648053968457728,peppa1963 rag_HLVS,ArrugoCristina,1129789421904158721,"@rag_HLVS Se lo van a cargar al pobre títere de Alberto Fernández.... ""Que parezca un accidente......"" #ArrugoCristina #BuenSabado",1,1
4,1129796456632729600,Sat May 18 17:10:40 +0000 2019,"RT @ProfetaArgen: Macrista: Cristina tiene que bajarse, que no sea soberbia Also Macrista: #ArrugoCristina",False,False,Fede Schoeller,302339856,ProfetaArgen,ArrugoCristina,1129764924924878850,"Macrista: Cristina tiene que bajarse, que no sea soberbia Also Macrista: #ArrugoCristina",294,74


In [13]:
tweets_df.to_csv('./csvs/%s_tweets.csv' % collection, index=False, encoding='utf8')

In [14]:
users_df.description = users_df.description.astype(str)

In [15]:
users_df.description.fillna("")
users_df.description = users_df.description.apply(clean_text)

In [16]:
users_df.shape

(36858, 11)

In [17]:
users_df.head()

,screen_name,location,followers_count,statuses_count,description,friends_count,favourites_count,created_at,id,name,verified
0,silvis2206,Buenos Aires,89,3129,"“Nada hay absoluto, todo se cambia, todo se mueve, todo revoluciona, todo vuela y se va.”",713,8113,Sun Nov 21 15:54:52 +0000 2010,218159646,silvina,False
1,Tonga_Arman80,Argentina,393,13491,Lo importante es la familia!!! La patria es el otro!!! #TropaK,497,58295,Mon May 17 16:27:25 +0000 2010,144904500,Tonga,False
2,Tewhok,None,3015,58461,Cenador Vitalicio Decapitalista,4919,7648,Sat Jun 18 11:44:51 +0000 2011,319594285,Guillermo Sordo,False
3,SilviaMam1,None,675,46338,None,708,18632,Fri Mar 03 12:57:13 +0000 2017,837648053968457728,Silvia Berth,False
4,avaro_schoeller,María Juana y Santa Fe,171,1113,"Inconstancia, inconciencia y lealtad.",374,346,Sat May 21 01:08:28 +0000 2011,302339856,Fede Schoeller,False


In [18]:
users_df.to_csv('./csvs/%s_users.csv' % collection, index=False, encoding='utf8')